In [0]:
# Célula para limpeza (execute uma vez)
try:
    dbutils.fs.unmount("/mnt/ecommerce")
    print("Ponto de montagem /mnt/ecommerce removido.")
except:
    print("Nenhum ponto de montagem /mnt/ecommerce encontrado para remover.")

In [0]:

storage_account_name = "dlsecommerce"


storage_account_key = "<key aki>"

# Lista dos containers 
containers_para_montar = ["raw", "bronze", "silver", "gold"]

print("Iniciando a montagem dos containers...")

# --- Loop para Montar cada Container ---
for container in containers_para_montar:
    mount_point = f"/mnt/{container}"
    source_url = f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/"

    # Primeiro, verificamos se o ponto de montagem já não existe
    if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
        try:
            # Se não existe, tentamos montar
            dbutils.fs.mount(
                source = source_url,
                mount_point = mount_point,
                extra_configs = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
            )
            print(f"SUCESSO: Container '{container}' foi montado em '{mount_point}'")
        except Exception as e:
            print(f"ERRO ao montar o container '{container}': {e}")
    else:
        # Se já existe, apenas informamos
        print(f"AVISO: O ponto de montagem '{mount_point}' já existe. Nenhuma ação necessária.")

print("\nVerificação final dos pontos de montagem:")
# Exibe todos os pontos de montagem ativos para confirmação
display(dbutils.fs.mounts())